<a href="https://colab.research.google.com/github/PeterDeWeirdt/defense_predictor/blob/main/defense_predictor_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DefensePredictor: A Machine Learning Model to Discover Novel Prokaryotic Immune Systems

Example usage of the [defense_predictor](https://github.com/PeterDeWeirdt/defense_predictor) python pacakge. DefensePredictor is a machine-learning model that leverages embeddings from a protein language model, [ESM2](https://github.com/facebookresearch/esm), to classify proteins as anti-phage defensive.

We reccomend using a GPU runtime for this notebook to accelerate predictions.
<br>
<br>
**If you're looking to run predictions for your own genome:** simply change the `ftp_link_base` below to the ftp webpage for your assembly of interest. You can find the ftp webpage for you genome by visiting the genome assembly webpage on NCBI and clicking the link that says "FTP" at the top of the page.

After changing the `ftp_link_base` variable click "Runtime" -> "Run all" to run predictions (should take ~10 minutes total). Once your predictions are done running a csv will be downloaded that has your predictions.

If you run into any problems please [raise an issue](https://github.com/PeterDeWeirdt/defense_predictor/issues/new) at the DefensePredictor github page.

In [1]:
ftp_link_base = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/'

## Create a virutal environment and install defense_predictor

In [2]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.3 MB/s eta 0:00:00


In [3]:
!virtualenv defense_predictor

created virtual environment CPython3.11.12.final.0-64 in 945ms
  creator CPython3Posix(dest=/content/defense_predictor, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.0.1, setuptools==78.1.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [4]:
!source defense_predictor/bin/activate;pip install defense_predictor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!source defense_predictor/bin/activate;defense_predictor_download

## Run DefensePredictor

Modify the `ftp_link_base` with your assembly of interest to get predictions

In [6]:
assembly_num = ftp_link_base.split('/')[-2]
ft_link = f'{ftp_link_base}{assembly_num}_feature_table.txt.gz'
fna_link = f'{ftp_link_base}{assembly_num}_cds_from_genomic.fna.gz'
faa_link = f'{ftp_link_base}{assembly_num}_protein.faa.gz'

We can download the necessary files directly from the ftp directory

In [7]:
input_file_list = list()
for link in [ft_link, fna_link, faa_link]:
  !wget {link}
  f_name = link.split('/')[-1]
  !gunzip {f_name}
  input_file_list.append(f_name[:-3])

--2025-05-02 15:22:43--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_feature_table.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231979 (227K) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_feature_table.txt.gz’

GCF_000005845.2_ASM 100%[===================>] 226.54K   263KB/s    in 0.9s    

2025-05-02 15:22:45 (263 KB/s) - ‘GCF_000005845.2_ASM584v2_feature_table.txt.gz’ saved [231979/231979]

--2025-05-02 15:22:45--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.

We can now run `defense_predictor` on our files and save outputs to the `files` folder. The predicted probability and log-odds of defense for each protein are in the columns `defense_probability` and `defense_log_odds`, respectively. We reccomend using a log-odds cutoff of `7.2` to confidently call proteins predicted defensive.

In [8]:
!source defense_predictor/bin/activate;defense_predictor \
--ncbi_feature_table {input_file_list[0]} \
--ncbi_cds_from_genomic {input_file_list[1]} \
--ncbi_protein_fasta {input_file_list[2]} \
--output {assembly_num}_defense_predictor_output.csv

Reading data
Getting ESM2 encodings
100% 347/347 [03:31<00:00,  1.64it/s]
Calculating remaining features
Making predictions


In [9]:
from google.colab import files

In [10]:
files.download(f'{assembly_num}_defense_predictor_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>